In [1]:
import os
from pathlib import Path
import sys
import os
import _pickle as pickle
import json
import copy 
import pandas
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.backends.backend_pdf import PdfPages
import csv
import json
from collections import Counter
import statistics 
import plotly.graph_objects as go
import math

In [2]:
from itertools import combinations
from scipy.stats import ttest_ind
import decimal

# create a new context for this task
ctx = decimal.Context()

# 20 digits should be enough for everyone :D
ctx.prec = 20

def float_to_str(f):
    """
    Convert the given float to a string,
    without resorting to scientific notation
    """
    d1 = ctx.create_decimal(repr(f))
    return format(d1, 'f')

In [3]:
with open('../../responseJudgement.json') as json_file:
    judgements = json.load(json_file)
    
with open('../../responseJudgement_sugg.json') as json_file:
    suggJudgements = json.load(json_file)

In [4]:
len(judgements)

15417

In [5]:
len(suggJudgements)

1014

In [6]:
len(judgements)

15417

In [7]:
cleanPath = '../../clean/'

In [8]:
responseSet = {}

for root, dirs, files in os.walk(cleanPath):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    currFileName = data['currentFileName']
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        uniqueStr = 'fileIndex=' + str(context['fileIndex']) + '&&contextLen=' + str(context['contextLen']) + '&&userResponse=' + context['userResponse'].strip()
                        
                        if(uniqueStr in responseSet):
                            continue
                        
                        else:
                            
                            foundMatches = [x for x in judgements if x['Input.contextLen'] == str(context['contextLen']) and x['Input.fileIndex'] == str(context['fileIndex'])]
                            foundJudgement = [x for x in foundMatches if x['Input.fileName'] == currFileName and x['Input.localIndex'] == str(context['localIndex'])]
                            
                            if(len(foundJudgement) > 0):
                                responseSet[uniqueStr] = foundJudgement
                        
                except Exception as e: print(e)

In [9]:
len(responseSet)

5139

In [10]:
suggSet = {}

for root, dirs, files in os.walk(cleanPath):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    currFileName = data['currentFileName']
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        suggestions = context['suggestions']
                        
                        for sugg in suggestions:
                            uniqueStr = 'fileIndex=' + str(context['fileIndex']) + '&&contextLen=' + str(context['contextLen']) + '&&userResponse=' + sugg.strip()

                            if(uniqueStr in responseSet or uniqueStr in suggSet):
                                continue

                            else:

                                foundMatches = [x for x in suggJudgements if x['Input.contextLen'] == str(context['contextLen']) and x['Input.fileIndex'] == str(context['fileIndex'])]
                                foundJudgement = [x for x in foundMatches if x['Input.fileName'] == currFileName and x['Input.localIndex'] == str(context['localIndex'])]

                                if(len(foundJudgement) > 0):
                                    suggSet[uniqueStr] = foundJudgement
                        
                except Exception as e: print(e)

In [11]:
len(suggSet)

458

In [12]:
context

{'providedConversation': ['I grew up with 8 older siblings, pretty crazy right? I think it has something to do with a fertility factor, but I have no idea what that is really, can you explain it?',
  'Whoa 8!!! Fertility factor has to do with how likely a person is to have children. ',
  'Oh I see, I guess my mother had quite a high fertility factor then.',
  'Yes she did.  Fertility factors  is based on a lot of things. ',
  'Alright, can you elaborate on what some of those things are?'],
 'suggestions': [],
 'userResponse': 'Fertility can be based on genetics, diet and contraceptives among others!',
 'contextLen': 3,
 'fileIndex': 27,
 'localIndex': 15}

In [13]:
Qualtype = 'Quality'
metric = {
    'Answer.' + Qualtype +'.1 (Very Poor)': 1,
    'Answer.' + Qualtype +'.2 (Poor)': 2,
    'Answer.' + Qualtype +'.3 (Okay)': 3,
    'Answer.' + Qualtype +'.4 (Good)': 4,
    'Answer.' + Qualtype +'.5 (Very Good)': 5
}

In [14]:
scores1 = []

for root, dirs, files in os.walk(cleanPath + "Setting 1"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        
                        userResponse = context['userResponse']
                        fileIndex = context['fileIndex']
                        contextLen = context['contextLen']
                        
                        identifierStr = 'fileIndex=' + str(fileIndex) + '&&contextLen=' + str(contextLen) + '&&userResponse=' + userResponse.strip()
                        
                        responseJudgements =  responseSet[identifierStr]
                        
                        currFeedback = []
                        for feedback in responseJudgements:
                            for key in feedback:
                                
                                if('Answer.' + Qualtype in key and feedback[key] == 'true'):
                                    currFeedback.append(metric[key])
                        
                        scores1.append(statistics.mean(currFeedback))
                            
                        
                    
                except Exception as e: print(e)                 

In [15]:
scores2 = []

for root, dirs, files in os.walk(cleanPath + "Setting 2"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        
                        userResponse = context['userResponse']
                        fileIndex = context['fileIndex']
                        contextLen = context['contextLen']
                        
                        identifierStr = 'fileIndex=' + str(fileIndex) + '&&contextLen=' + str(contextLen) + '&&userResponse=' + userResponse.strip()
                        
                        responseJudgements =  responseSet[identifierStr]
                        
                        currFeedback = []
                        for feedback in responseJudgements:
                            for key in feedback:
                                
                                if('Answer.' + Qualtype in key and feedback[key] == 'true'):
                                    currFeedback.append(metric[key])
                        
                        scores2.append(statistics.mean(currFeedback))
                            
                        
                    
                except Exception as e: print(e)                 

In [16]:
scores3 = []

for root, dirs, files in os.walk(cleanPath + "Setting 3"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        
                        userResponse = context['userResponse']
                        fileIndex = context['fileIndex']
                        contextLen = context['contextLen']
                        
                        identifierStr = 'fileIndex=' + str(fileIndex) + '&&contextLen=' + str(contextLen) + '&&userResponse=' + userResponse.strip()
                        
                        responseJudgements =  responseSet[identifierStr]
                        
                        currFeedback = []
                        for feedback in responseJudgements:
                            for key in feedback:
                                
                                if('Answer.' + Qualtype in key and feedback[key] == 'true'):
                                    currFeedback.append(metric[key])
                        
                        scores3.append(statistics.mean(currFeedback))
                            
                        
                    
                except Exception as e: print(e)                 

In [17]:
scores4 = []

for root, dirs, files in os.walk(cleanPath + "Setting 4"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        
                        userResponse = context['userResponse']
                        fileIndex = context['fileIndex']
                        contextLen = context['contextLen']
                        
                        identifierStr = 'fileIndex=' + str(fileIndex) + '&&contextLen=' + str(contextLen) + '&&userResponse=' + userResponse.strip()
                        
                        responseJudgements =  responseSet[identifierStr]
                        
                        currFeedback = []
                        for feedback in responseJudgements:
                            for key in feedback:
                                
                                if('Answer.' + Qualtype in key and feedback[key] == 'true'):
                                    currFeedback.append(metric[key])
                        
                        scores4.append(statistics.mean(currFeedback))
                            
                        
                    
                except Exception as e: print(e)                 

In [18]:
scores5 = []

for root, dirs, files in os.walk(cleanPath + "Setting 5"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        
                        userResponse = context['userResponse']
                        fileIndex = context['fileIndex']
                        contextLen = context['contextLen']
                        
                        identifierStr = 'fileIndex=' + str(fileIndex) + '&&contextLen=' + str(contextLen) + '&&userResponse=' + userResponse.strip()
                        
                        responseJudgements =  responseSet[identifierStr]
                        
                        currFeedback = []
                        for feedback in responseJudgements:
                            for key in feedback:
                                
                                if('Answer.' + Qualtype in key and feedback[key] == 'true'):
                                    currFeedback.append(metric[key])
                        
                        scores5.append(statistics.mean(currFeedback))
                            
                        
                    
                except Exception as e: print(e)                 

In [19]:
settings = {
    '1': scores1,
    '2': scores2,
    '3': scores3,
    '4': scores4,
    '5': scores5
}

for list1, list2 in combinations(settings.keys(), 2):
    t, p = ttest_ind(settings[list1], settings[list2], equal_var = False)
    print (list1, list2, float_to_str(p))

1 2 0.4491044844989873
1 3 0.00017264744213950088
1 4 0.00000000000000000001853851916370419
1 5 0.000000000003011886066412356
2 3 0.00000000003975131258029085
2 4 0.00000000000000000000000000000000000000000000000000000000000000000000000000000063887950009069
2 5 0.00000000000000000000000000000000389152663778181
3 4 0.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001431394343829798
3 5 0.00000000000000000000000000000000000000000000000000000000020063475296497167
4 5 0.010879488413306824


In [21]:
colors = ['#FAEBD7', '#FFA07A', '#B0C4DE', '#AFEEEE', '#8FBC8F']

x = ['Setting 1 (H)', 'Setting 2', 'Setting 3', 'Setting 4', 'Setting 5']
y = [scores1, scores2, scores3, scores4, scores5]

yErorr = [np.std(z)/math.sqrt(len(z)) for z in y] 

y = [round(statistics.mean(z), 2) for z in y]

fig = go.Figure()
fig.add_trace(go.Bar(
    name='Setting',
    x=x, y=y, text=y, textposition='inside',insidetextanchor='middle',
    marker_color=colors,
    marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.8,
    error_y=dict(type='data', array=yErorr)
))

fig.update_layout(
    title={
        'text': 'Overall response quality',
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(range=[3, 4.2])


fig.show()